# FLOWCELL | OPERATION

[AAAA-MM-JJ] Suivi des opérations. Renseigner les noms de la _flowcell_ et de l'exérience (informations dans la SampleSheet, communiquées par courriel du CQGC ou récupérées de BaseSpace).

Cette ligne et les suivantes de cette cellule sont à titre informatives et peuvent être supprimées du journal.

0. Mise-en-place de l'environnement de travail
    1. Déconvoluer et convertir les BCLs en FASTQs `dragen_bcl-convert_watcher.sh`
1. Préparer les cas `run_pipeline_prag.sh`
    1. Collecter les informations sur les familles
    2. Téléverser les FASTQs sur BaseSpace
    3. Générer le fichier "emg_batch_manifest.csv"
2. Glisser-déposer dans Emedgene le fichier "emg_batch_manifest.csv"
3. Collecter les metriques
4. Archiver les résultats
5. Nettoyer


## DÉPANNAGE

- Afin de connecter les informations génétiques des familles (issues de Nanuq) aux phénotypes (termes HPOs contenus dans Phenotips), il est impératif que les deux champs EP (Établissement Public) et MRN (Medical Record Number) soient bien renseignés dans les deux systèmes par les collègues. Sinon, il faut obtenir les informations de Phenotips (identifiants PID) par "Camille Varin-Tremblay (HSJ)" <camille.varin-tremblay.hsj@ssss.gouv.qc.ca>.
- Si le séquençage est réalisé avec le NovaSeq6000, il ne sera malheureusement pas possible de récupérer les métriques avant de créer les cas dans Emedgene. Dans ce cas, il faut s'assurer que le labo a demandé au NovaSeq6000 de faire la déconvolution sur BaseSpace et passer à l'étape 3, "créer les cas sur Emedgene", une fois que les FASTQs ont été générés (suivi sur l'onglet [Analyses](https://chusj.cac1.sh.basespace.illumina.com/analyses)). 
- Dans la situation où la déconvolution n'a pas lieu sur BaseSpace, il faut déconvoluer les BCLs avec les serveurs DRAGEN du CQGC et envoyer les FASTQs résultants sur BaseSpace (passer à l'étape 2, "Téléverser les FASTQs sur BaseSpace").


In [ ]:
## 0. Mise en place de l'environnement de travail
# ssh ${USER}@10.128.80.26

screen -S prag
conda activate CQGC-utils

export FC=""
export XP=""
a=($(echo ${FC} | tr '_' '\n'))
export FC_SHORT="${a[1]}_${a[2]}"
export BASEDIR="/mnt/spxp-app02/staging/hiseq_raw/${a[1]}"
export WORKDIR="/mnt/spxp-app02/staging2/dragen"

## 1. Préparer les cas
## Normalement, bcl-convert est automatique par cronjob

cd ${WORKDIR}
echo "Waiting to demux..."
until [ -d "${WORKDIR}/${FC}/1.fastq" ]; do
    # bash /staging2/soft/CQGC-utils/Analysis.dragen_bcl-convert/scripts/dragen_bcl-convert_watcher.sh | tee -a /staging2/dragen/dragen_bcl-convert_watcher.log
    bash /staging2/soft/CQGC-utils/Analysis.dragen_bcl-convert/scripts/dragen_bcl-convert_watcher.sh  2>&1 >> /staging2/dragen/dragen_bcl-convert_watcher.log
    printf '.'
    sleep 900
done
cd ${WORKDIR}/${FC}
bash /staging2/soft/CQGC-utils/Analysis.pipeline_illumina/run_pipeline_prag.sh ${FC} 2>&1 | tee run_pipeline_prag.log
echo "delta"


In [ ]:
## 2. Glisser-déposer dans Emedgene le fichier "emg_batch_manifest.csv"

In [ ]:
## 3. Collecter les metriques

mkdir ${WORKDIR}/${FC_SHORT} && cd "$_"
python /staging2/soft/CQGC-utils/Analysis.pipeline_illumina/emg_collect_samples_metrics.py ${FC}

In [ ]:
## 4. Archiver les résultats

ssh hien@narval.calculquebec.ca

samples=()
total_samples=${#samples[@]}

cd ${HOME}/projects/ctb-rallard/COMMUN/PRAGMatIQ-EMG
cp archive_PRAGMatIQ.log archive_PRAGMatIQ.log0

count=1
for sample in ${samples[@]}; do
    echo "Processing sample ${count}/${total_samples}, ${sample}"
    aws s3 --profile emedgene cp s3://cac1-prodca-emg-auto-results/CHU_Sainte_Justine/${sample}/ ./archives/${sample} --recursive
    echo ${sample} $( date +'%Y-%m-%d %T' ) >> archive_PRAGMatIQ.log
    ((count++))
done

In [ ]:
# 6. Nettoyer
# Avant d'effacer le répertoire de la run, otifier le laboratoire qui doit sauvegarder quelqeues métriques

`rm -R ${BASEDIR}/${FC}`

## Logs